In [1]:
import os 
import re
import time
import shutil
import requests
import natsort
import json
import random
from glob import glob
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from PIL import Image, ImageOps

import tifffile
import rasterio
import geopandas as gpd # geo 데이터프레임 만들기 위함
from pyidw import idw # 보간법 실행
from osgeo import gdal # 래스터 자르기 anaconda로 설치
from rasterio.plot import show # 래스터 시각화
from shapely.geometry import Point
from rasterio.features import geometry_mask
from rasterio.transform import from_origin

### asos 크롤링 funcion

In [2]:
def asos_crawling(date,locn):
    """ 
    - asos 기상 데이터를 크롤링
    - 최대 5번까지 시도하는 방식
    """
    url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList' # 크롤링할 주소 

    startDt=datetime.strptime(date[:8], '%Y%m%d') # 시작날짜
    startHh = datetime.strptime(date[8:], '%H')   # 시작시간
    endHh = (startHh + timedelta(hours=1))        # 시작날짜 + 1 (23같은경우 00이 되게 하기위해서 timedelta 이용)
    if(endHh.strftime('%H:%M:%S').split(':')[0]=='00'):endDt=(startDt + timedelta(days=1))
    else:endDt=startDt # endhh==00이면 시작날짜와 종료날짜가 달라야함. 22일 23시와 23일 00시 이런식.

    startDt=startDt.strftime('%Y%m%d')
    startHh=startHh.strftime('%H:%M:%S').split(':')[0]
    endDt=endDt.strftime('%Y%m%d')
    endHh=endHh.strftime('%H:%M:%S').split(':')[0]

    # Servicekey list
    # pXv2lwkQOlbcBekMvEjur8cGaDpQb84Upv6ZboITLKGudf2//PqPjL6QABwMtAhzNilIwndbgQdo9lBTUwy3XA==
    # 1tZqzN7UEB+yrAZ61++roasr+iAGFTX9QhDLFqpYOw7oDYHKMhRjCsk5jy8YAQz+xBdStnCvi4rQ/0SITX2cEg==
    # 3imQf/ygL+vTqRcXZ19hAwVhJhVDxZ2yRGtaRQPk/F3rFSVB2Kvu7LFfoGVhB4rYfTVk2kILGAhhJvmu9kQUzA==  : 아빠 
    params ={'serviceKey' : '1tZqzN7UEB+yrAZ61++roasr+iAGFTX9QhDLFqpYOw7oDYHKMhRjCsk5jy8YAQz+xBdStnCvi4rQ/0SITX2cEg==', #AuthenticationKey
                'pageNo' : '1',
                'numOfRows' : '10',
                'dataType' : 'JSON', 
                'dataCd' : 'ASOS', 
                'dateCd' : 'HR', 
                'startDt' : startDt, #startdate
                'startHh' : startHh, #starttime
                'endDt' : endDt, # end date
                'endHh' : endHh, # end time
                'stnIds' : locn 
            }
    for i in range(5):  # 최대 5번까지 시도
        try:
            response = requests.get(url, params=params,verify=False)
            #print(response.url) # 요청 url 출력하기 
            try:
                json_obj = json.loads(response.content)
                try:
                    json_obj=json_obj["response"]["body"]["items"]["item"][0]
                    times=json_obj['tm']
                    humidity=json_obj['hm']
                    windspeed=json_obj['ws']
                    rain=json_obj['rn']
                    temp=json_obj['ta']              
                    return times, humidity, windspeed, rain, temp
                except:
                    #if(json_obj["response"]['header']['resultMsg']=='NO_DATA'):
                        #print("No_Data")
                        #return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
                    #else:
                        #print("Retry")
                    print("Retry1")
                    print(json_obj)
                    time.sleep(5)
            except:
                print(response.content)
                try:
                    json_obj = json.loads(response.content)
                    print(response,json_obj)
                    print("Retry2")
                    time.sleep(5)
                except json.JSONDecodeError:
                    time.sleep(2)
                    continue
            # 네트워크 연결 끊겼을 때.    
        except (requests.exceptions.HTTPError, requests.exceptions.ConnectionError) as e:
            print(f'오류 발생 재시도')
            time.sleep(2)
        #print(response.content)
        # 현재는 시간,습도,풍속,강수량,기온 
    return np.nan,np.nan,np.nan,np.nan,np.nan

### 기상 데이터 수집 function

In [3]:
def crawl_data(data_n,o_data,method):
    """
    method==1 
        - api를 이용해서 직접 크롤링 하는 방식->새로운 데이터를 얻을 수 있지만, api 불안정하여 시간이 오래걸림
        - 데이터 크롤링
        - 산불발생 데이터에서 산불 발생 날짜 가져옴
        - 산불 발생 날짜에 대해서 전 지역 asos 데이터를 크롤링
        - 한번 크롤링 할 때 한 asos 지점에 대한 기상데이터를 가져오면서 합침 
        - 최종으로 산불발생 개수 만큼 csv가 생성되고, 각 csv에는 산불발생시점 전 지역에 대한 기상데이터 존재 
    
    method==2
        - 기존에 수집한 asos 시간별 데이터에서 가져옴->최신화 어렵고, 용량 차지하는데 크롤링보다는 빠르고 정확
    
    
    """
    print("Start crawling asos clmate data")
    filepath=f"../data/data_set({data_n})/climate_data/"
    os.makedirs(filepath, exist_ok=True)
    
    if((len(glob(os.path.join(filepath, "*.csv"))))==len(o_data)):
        print("-->Already existed")
        print("---------------")
    else:
        loc_data=pd.read_csv("../data/aws_loc_list.csv") # asos 지점과 경위도 데이터 
        loc_list=loc_data['지점번호']
        result=[]
        
        if(method==1):
            for i in tqdm(range(len(o_data))):
                tmp=pd.DataFrame(columns=['num','loc_info', 'lon','lat','time','humidity','wind_sp','rainfall','temp'])
                for j in range(len(loc_list)):
                    new_data=[i,loc_list[j],loc_data['lon'][j],loc_data['lat'][j]]
                    new_data.extend(asos_crawling(o_data['input'][i],loc_list[j])) 
                    tmp = pd.concat([tmp, pd.DataFrame([new_data], columns=tmp.columns)], ignore_index=True)
                tmp.to_csv(f"../data/data_set({data_n})/climate_data/data_{i}.csv",encoding='cp949',index=False) 
            
        if(method==2):
            for i in tqdm(range(len(o_data))):
                o_data['input'] = o_data['input'].astype(str)
                year = o_data['input'][i][:4]
                
                df_asos_hr = pd.read_csv(f'../data/ASOS_Hr/ASOS_Hr_{year}.csv', low_memory=False)
                
                date=datetime.strptime(o_data['input'][i], "%Y%m%d%H")
                date = date.strftime("%Y-%m-%d %H:%M")
                df_asos_hr = df_asos_hr[df_asos_hr['일시'] == date]
                df_asos_hr = df_asos_hr.reset_index(drop=True)

                df_asos_hr['num'] = i
                df_asos_hr['longitude'] = [loc_data[loc_data['지점번호']==df_asos_hr['지점'][i_asos_hr]]['lon'].values[0] if loc_data[loc_data['지점번호']==df_asos_hr['지점'][i_asos_hr]]['lon'].shape[0]!=0 else np.nan for i_asos_hr in range(df_asos_hr.shape[0])]
                df_asos_hr['latitude'] = [loc_data[loc_data['지점번호']==df_asos_hr['지점'][i_asos_hr]]['lat'].values[0] if loc_data[loc_data['지점번호']==df_asos_hr['지점'][i_asos_hr]]['lat'].shape[0]!=0 else np.nan for i_asos_hr in range(df_asos_hr.shape[0])]
                df_asos_hr = df_asos_hr[['num', '지점', 'longitude', 'latitude', '일시', '습도(%)', '풍속(m/s)', '강수량(mm)', '기온(°C)']]
                df_asos_hr.columns=['num','loc_info','lon','lat','time','humidity','wind_sp','rainfall','temp']
                df_asos_hr.to_csv(f'../data/data_set({data_n})/climate_data/data_{i}.csv',encoding='cp949',index=False)
                
                
        """
        최종적으로 수집한 각 산불별 기상데이터를 하나로 합치는 과정
        """
        for i in range(len(o_data)):
            tmp=pd.read_csv(f"../data/data_set({data_n})/climate_data/data_{i}.csv",encoding='cp949')
            result.append(tmp)        
        weather_data = pd.concat(result,ignore_index=True)
        weather_data.to_csv(f"../data/data_set({data_n})/final_climate_data.csv",encoding='cp949',index=False)
        print("-->Complete")
        print("---------------")
        


### 기상 데이터 보간 function

In [4]:
def interpolate_climate(data_n,o_data):
    
    """
    - 전국의 모든 지역에 대한 기상데이터가 존재하지 않기 때문에,
    asos의 모든 지점으로 부터 얻은 기상데이터를 이용하여, 각 feature별로 데이터를 보간(역거리가중법 / idw보간)
    - 보간시 하나의 feature라도 없으면 전부 drop( 결측치 )
    - 한 지점이라도 살아있으면 보간이 되서 우선 냅둠 -
    # -->어떤 산불은 지점 여러개로 보간된 데이터, 어떤건 지점 한,두개로 보간된 데이터
    
    """
    
    print("Start climate interpolation")
    filepath=f"../data/data_set({data_n})/interpolate_climate/"
    os.makedirs(filepath, exist_ok=True)
    
    if(len(glob(filepath + '/*')))!=4: # 4개의 폴더가 있어야 함

        features=['humidity','wind_sp','rainfall','temp']
        
        os.makedirs(filepath+features[0], exist_ok=True)
        os.makedirs(filepath+features[1], exist_ok=True)
        os.makedirs(filepath+features[2], exist_ok=True)
        os.makedirs(filepath+features[3], exist_ok=True)

        weather_data=pd.read_csv(f"../data/data_set({data_n})/final_climate_data.csv",encoding='cp949')
        weather_data.columns=['num','loc_info','lon','lat','time','humidity','wind_sp','rainfall','temp']
        weather_data.dropna(subset=['time'], inplace=True) # time이 null이면 데이터가 정상적으로 크롤링되지 않은 것임. 
        weather_data['rainfall']=weather_data['rainfall'].fillna(0) # 강수가 비어있는건 0으로 채움 

        for i in tqdm(range(len(o_data))):
            tmp=weather_data[weather_data['num']==i]
            tmp=tmp.dropna()
            # 결측치 전부 drop 후에 데이터가 없으면 보간하지 않음
            if(len(tmp)==0):
                print("There is no data")
                continue
            tmp.drop(['num','loc_info','time'],axis=1,inplace=True)
            tmp = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp.lon, tmp.lat))
            tmp.to_file(f'data{i}.shp')
            
            for j in range(len(features)):
                idw.idw_interpolation(
                    input_point_shapefile=f'data{i}.shp', # 보간하고자 하는 shp 파일 
                    extent_shapefile="../data/gw_boundary/boundary.shp", # 경계 shp 파일(현재 강원도)
                    column_name=features[j], # 보간하고자 하는 feature 이름. 
                    power=2, # 거리 가중치 계수 
                    search_radious=8, # 검색하고자 하는 범위 
                    output_resolution=400, # 결과물 해상도 
                )
                image=rasterio.open(f"data{i}_idw.tif")
                image=pd.DataFrame(image.read(1))
                image.to_csv(f"{filepath}{features[j]}/data{i}_idw.csv",encoding='cp949')
                os.remove(f"data{i}_idw.tif")
            
            os.remove(f"data{i}.shp")
            os.remove(f"data{i}.cpg")
            os.remove(f"data{i}.dbf")
            os.remove(f"data{i}.shx")
            print("-->Complete")
            print("---------------")
    else:
        print("-->Already existed")
        print("---------------")

### 산불 발생 위치 탐색 function

In [5]:
def find_fireloc(data_n,o_data):
    
    """
    다른 기상데이터는 필요없고, 산불 발생 위치의 기상데이터만 필요하기 때문에 산불 발생 위치를 찾는 코드
    이미 산불 발생 위치의 경위도는 알고 있지만, 보간된 기상데이터에서 산불 발생 위치의 데이터를 가져오기 위해선
    아래와 같이 생성한 shp를 tif로 변환하면서, 강원도 밖은 32767 강원도 내부는 -9999, 산불 발생 위치는 624로 채움
    --> 추후,다른 부분의 기상이 필요하지 않다면, 기상청만큼 신뢰성 있는 사이트에서 기상데이터 한개만을 가져오는 방식이 훨씬 효율적
    """
    
    print("Start find fire loc")
    filepath=f"../data/data_set({data_n})/fire_loc/"
    os.makedirs(filepath, exist_ok=True)
    
    if(len(glob(filepath + '/*')))==0:
        for i in tqdm(range(len(o_data))):
            tmp=pd.DataFrame(o_data.iloc[i])
            tmp=tmp.T
            tmp = gpd.GeoDataFrame(tmp, geometry=gpd.points_from_xy(tmp.lon, tmp.lat))
            tmp.drop(['date','time','lon','lat','input'],axis=1,inplace=True)
            tmp.to_file('fire_data.shp')

            shp_path  = "fire_data.shp" # 현재 shp파일 이름 
            boundary_path='../data/gw_boundary/boundary.shp'
            tif_path  = f"fire_data.tif" # 만들고자 하는 tif파일 이름

            driver = gdal.GetDriverByName('GTiff')
            # 산불 shp 파일을 읽어오기
            shp_datasource = gdal.OpenEx(shp_path, gdal.OF_VECTOR)
            # 경계shp 파일을 읽어오기
            boundary_datasource = gdal.OpenEx(boundary_path, gdal.OF_VECTOR)
            # tif 파일 생성
            tif_datasource = driver.Create(tif_path, 400, 278, 1, gdal.GDT_Float32)
            # 좌표계 설정
            tif_datasource.SetProjection(shp_datasource.GetProjection())
            # 강원도 경계값 가져옴 
            boundary = gpd.read_file(boundary_path)
            xmin, ymin, xmax, ymax = boundary.total_bounds
            # 강원도 경계값 가져온걸 위에서 설정한 400,278 즉 액셀 파일 크기형태로 설정 
            tif_datasource.SetGeoTransform((xmin, (xmax-xmin)/400, 0, ymax, 0, -(ymax-ymin)/278))
            band = tif_datasource.GetRasterBand(1)
        #   산불 난 지점 외에는 전부 32767으로 설정
            band.Fill(32767)
            band.SetNoDataValue(32767)
            # 강원도 내부는 -9999로 채우기
            gdal.RasterizeLayer(tif_datasource, [1], boundary_datasource.GetLayer(), burn_values=[-9999], options=["ALL_TOUCHED=TRUE"])
            # 산불 발생 위치는 624로 설정
            gdal.RasterizeLayer(tif_datasource, [1], shp_datasource.GetLayer(), burn_values=[624]) 
            #gdal.RasterizeLayer(tif_datasource, [1], boundary_datasource.GetLayer(), burn_values=[32767])
            shp_datasource = None
            tif_datasource = None
            tif_file = rasterio.open(f"fire_data.tif") 
            data = tifffile.imread(f"fire_data.tif")    
            data=pd.DataFrame(data)
            data.to_csv(f"{filepath}fire_data{i}.csv")
            tif_file.close()
            
        # 필요없는 파일 삭제
        os.remove(f"fire_data.shp")
        os.remove(f"fire_data.cpg")
        os.remove(f"fire_data.dbf")
        os.remove(f"fire_data.shx")
        os.remove(f"fire_data.tif")
        print("-->Complete")
        print("---------------")
    else:
        print("-->Already existed")
        print("---------------")
        

#### 산불 발생 위치 기상데이터 수집 function

In [6]:
def get_climate_info(data_n,o_data):
    
    """
    - 보간한 기상데이터에서 산불발생위치의 기상 데이터만을 가져옴.
    """
    
    print("Start get climate information")
    filepath=f"../data/data_set({data_n})/train_{data_n}.csv"
    
    if os.path.isfile(filepath):
        print("Already existed")
        print("---------------")
    else:
        climate=[]
        for i in tqdm(range(len(o_data))):
            data=pd.read_csv(f"../data/data_set({data_n})/fire_loc/fire_data{i}.csv")
            index = data[data == 624].stack().index[0] # 산불발생위치를 찾음 
            temp=pd.read_csv(f"../data/data_set({data_n})/interpolate_climate/temp/data{i}_idw.csv")
            rain=pd.read_csv(f"../data/data_set({data_n})/interpolate_climate/rainfall/data{i}_idw.csv")
            hums=pd.read_csv(f"../data/data_set({data_n})/interpolate_climate/humidity/data{i}_idw.csv")
            wind=pd.read_csv(f"../data/data_set({data_n})/interpolate_climate/wind_sp/data{i}_idw.csv")
            
            r,c=index[0],int(index[1])
            tmp_data = [
                    [a, b, c, d]
                    for a, b, c, d in zip(
                        [temp.iloc[r][c]],
                        [rain.iloc[r][c]],
                        [hums.iloc[r][c]],
                        [wind.iloc[r][c]]
                    )
                ]
            climate.append(tmp_data[0])
        climate=pd.DataFrame(climate,columns=['기온','강수','습도','풍속'])
        o_data.drop('input',axis=1,inplace=True)
        trainfire=pd.concat([o_data,climate],axis=1,ignore_index=True)
        trainfire.columns=['date','time','lon','lat','temp','rainfall','humidity','windspeed']  
        trainfire.to_csv(filepath,index=False)
        print("-->Complete")
        print("---------------")

#### 산불 안난 트레인 셋 만들기 function

In [7]:
def make_random_dataset(start_year, end_year, num_samples,filepath): 
    
    """
    - 산불 난 데이터는 target==1로 실존 데이터이고, 산불 나지 않은 데이터는 target==0으로 임의로 만든 데이터 셋이다.
    - 데이터 셋을 만들 떄, 어떻게 만드냐에 따라 매우 달라진다.
    - 같은 날씨에 같은 지형이 들어가는 경우는 없어야 하며, 계절과 날짜 모두 랜덤으로 골고루게 되었다.
    - 데이터는 반드시 강원도 이내에 있어야 한다.
    - 현재 만든 방식은 
      1. 같은 지점을 총 5번씩 다른 날짜에 뽑았음--> 228*5=1140개의 데이터셋 만듦 
    """
    seasons = ['spring', 'fall', 'winter','else']
    season_weights = [0.6, 0.2, 0.1, 0.1]  # 비율을 설정합니다. 봄: 0.3, 여름: 0.15, 가을: 0.35, 겨울: 0.2
    
    min_latitude,max_latitude = 37.03353708,38.61370931
    min_longitude,max_longitude = 127.0950376, 129.359995

    tmp = []
    for _ in range(num_samples):
        year = random.randint(start_year, end_year)
        hour = random.randint(0, 23)
        season = random.choices(seasons, weights=season_weights)[0]
        
        if season == 'spring':
            month = random.randint(2, 5)
        elif season == 'fall':
            month = random.randint(11, 12)
        elif season == 'winter':
            month = random.randint(6, 10)
        else:
            month = random.randint(1, 1)
            
        day = random.randint(1, 28)
        minute,second=0,0
        time = pd.Timestamp(year, month, day, hour, minute, second)

        tmp.append(time)
        
    dates_df=pd.DataFrame(tmp,columns=['Date'])
    dates_df['date'] = pd.to_datetime(dates_df['Date']).dt.strftime('%Y%m%d')
    dates_df['time'] = pd.to_datetime(dates_df['Date']).dt.strftime('%H%M%S')
    
    dates_df["date"] = dates_df["date"].str.zfill(8)
    dates_df["time"] = dates_df["time"].str.zfill(6)
    
    dates_df.drop('Date',axis=1,inplace=True)
    #dates_df['Year'] = pd.to_datetime(dates_df['Date']).dt.year
    #dates_df['Month'] = pd.to_datetime(dates_df['Date']).dt.month
    #dates_df['Day'] = pd.to_datetime(dates_df['Date']).dt.day
    #dates_df.groupby('Month').count()  
    
    tmp=[]
    
    for _ in range(500): # 강원도 바깥에생성될 걸 대비핵서 넉넉하게 
        while True:
            latitude = random.uniform(min_latitude, max_latitude)
            longitude = random.uniform(min_longitude, max_longitude)
            coordinate = (latitude, longitude)
            if coordinate not in tmp:
                tmp.append((longitude, latitude))
                break
    df = pd.DataFrame(tmp, columns=['lon', 'lat'])
    #plt.scatter(df.lon,df.lat) # 전체분포가 올바른지 확인 

    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
    df.crs = "EPSG:4326" # 좌표계 설정 
    df.to_file(f'df.shp')
    df = gpd.read_file('df.shp')
    gangwon_df = gpd.read_file("../data/gw_boundary/boundary.shp")
    gangwon_boundary = gangwon_df.geometry.unary_union
    df = df[df.geometry.within(gangwon_boundary)][:228]
    df.reset_index(drop=True,inplace=True)
    os.remove('df.shp')
    df.drop('geometry',axis=1,inplace=True)
    df=pd.concat([df,df,df,df,df],axis=0,ignore_index=True)
    
    notfire=pd.concat([dates_df,df],axis=1)
    
    notfire['input'] = notfire['date'].astype(str)+notfire['time'].apply(lambda x: str(x)[:2]).str.zfill(2).astype(str)
    
    notfire.to_csv(filepath,index=False)

# Main code

In [8]:
def make_fire_dataset(data_n):
    filepath=f"../data/gangwon_{data_n}.csv"
    if os.path.isfile(filepath)==False: # nofire 데이터가 없을 경우 생성해야함. 
        make_random_dataset(2011, 2022, 1140,filepath)
    o_data=pd.read_csv(f"../data/gangwon_{data_n}.csv")
    print("#"*50)
    print(f"Make {data_n} dataset ")
    weather_data=crawl_data(data_n,o_data,2)
    interpolate_climate(data_n,o_data)
    find_fireloc(data_n,o_data)
    get_climate_info(data_n,o_data)
    dataset=pd.read_csv(f"../data/data_set({data_n})/train_{data_n}.csv")
    return dataset
        
        

In [13]:

fire_dataset=make_fire_dataset("fire")
fire_dataset['target']=1
nofire_dataset=make_fire_dataset("nofire")
nofire_dataset['target']=0
final_dataset=pd.concat([fire_dataset,nofire_dataset],axis=0,ignore_index=True)
final_dataset
final_dataset.to_csv("../data/train_data.csv",index=False)

##################################################
Make fire dataset 
Start crawling asos clmate data
-->Already existed
---------------
Start climate interpolation
-->Already existed
---------------
Start find fire loc
-->Already existed
---------------
Start get climate information
Already existed
---------------
##################################################
Make nofire dataset 
Start crawling asos clmate data
-->Already existed
---------------
Start climate interpolation
-->Already existed
---------------
Start find fire loc
-->Already existed
---------------
Start get climate information
Already existed
---------------


,date,time,lon,lat,temp,rainfall,humidity,windspeed,target
0,20110122,233500,128.869839,37.782030,-2.800407,0.00000,41.980458,3.389697,1
1,20110201,210700,128.852520,37.792612,1.070496,0.00000,54.996640,1.931208,1
2,20110226,164200,127.955332,37.379769,13.438066,0.00000,23.056794,1.431839,1
3,20110212,61800,127.879749,37.220708,-6.431949,0.01229,47.088550,1.842775,1
4,20110211,175000,128.128864,37.772593,1.532300,0.00000,55.537041,2.576569,1
...,...,...,...,...,...,...,...,...,...
2188,20180316,230000,129.034403,37.227534,-5.149554,0.00000,85.426590,0.777338,0
2189,20210111,210000,128.028175,37.885594,-8.744794,0.00000,69.356481,0.328502,0
2190,20140201,0,127.941031,37.395255,-0.415539,0.00000,72.227057,0.308869,0
2191,20180505,190000,128.302668,38.227089,21.923886,0.00000,25.347899,3.725006,0
